# A custom layer in Keras (that's just more, smaller layers)

## Entire models as layers

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.applications import ResNet50


In [2]:
input_shape = (243, 320, 1)

face_1 = Input(shape=input_shape)
face_2 = Input(shape=input_shape)

embedding_unit = ResNet50(
    input_shape=(243, 320, 3),
    weights='imagenet',
    include_top=False,
)
embedding_unit.trainable = False

pooling_layer = layers.GlobalAvgPool2D()
concatenation_layer = layers.Concatenate()

dense_layers = keras.Sequential(
    [
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid'),
    ]
)

embedding_1 = embedding_unit(
    concatenation_layer(3 * [face_1])
)
embedding_2 = embedding_unit(
    concatenation_layer(3 * [face_2])
)
both_embeddings = concatenation_layer(
    [
        pooling_layer(embedding_1),
        pooling_layer(embedding_2)
    ]
)
output = dense_layers(both_embeddings)

model = Model(inputs=[face_1, face_2], outputs=output)

Instructions for updating:
Colocations handled automatically by placer.


/Users/jan/miniconda3/envs/py37/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
from pathlib import Path
import numpy as np
from PIL import Image
import random

class FaceDataset(keras.utils.Sequence):
    
    def __init__(
        self,
        folder="/Users/jan/datasets/yale-face-database/",
        batch_size=32,
        n_channels=1,
        shuffle=True
    ):
        self.batch_size = batch_size
        self.root = Path(folder)
        self.files = list(self.root.glob("*"))
        self.images = np.stack(
            [np.array(Image.open(file)) for file in self.files]
        )[..., np.newaxis] / 255
        self.identities = np.array([int(str(file.stem).replace("subject", "")[:2]) for file in self.files])
    
    def __getitem__(self, index):
        'Generate one batch of data'
                
        indices_1 = np.random.randint(
            low=0, high=len(self.images), size=self.batch_size
        )
        indices_2 = np.random.randint(
            low=0, high=len(self.images), size=self.batch_size
        )
        
        X = [self.images[indices_1, ...], self.images[indices_2]]
        y = (self.identities[indices_1] == self.identities[indices_2]).astype(int)
        return X, y


    def __len__(self):
        return len(self.files) * 2 // self.batch_size


In [4]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0003),
    loss=keras.losses.BinaryCrossentropy()
)

model.fit_generator(
    generator=FaceDataset(batch_size=12),
    epochs=4,
    use_multiprocessing=True,
    workers=6
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/4
27/27 [==============================] - 82s 3s/step - loss: 0.3185
Epoch 2/4
27/27 [==============================] - 85s 3s/step - loss: 0.1449
Epoch 3/4
27/27 [==============================] - 85s 3s/step - loss: 0.0805
Epoch 4/4
27/27 [==============================] - 81s 3s/step - loss: 0.0588
